In [ ]:
! mkdir -p ~/aiffel/dlthon-minions/data
! ln -s ~/data/* ~/aiffel/dlthon-minions/data/

In [4]:
# 라이브러리 불러오기
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

import re
from collections import Counter
import sentencepiece as spm
from konlpy.tag import Okt
from konlpy.tag import Mecab
import csv 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [5]:
# 한글 폰트에 문제가 생겼을 때

# 한글 폰트 설치
!apt-get update -qq
!apt-get install -qq fonts-nanum

# 설치한 폰트를 matplotlib에서 사용할 수 있도록 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

# 나눔 폰트 경로 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

# 폰트 매니저에 폰트 추가
fm.fontManager.addfont(font_path)
plt.rc('font', family='NanumGothic')  # 폰트 설정

In [7]:
import os
import pandas as pd
train_data_path ="../data/conversations.csv"
train_data = pd.read_csv(train_data_path)
# 원본 데이터 저장
origin_data = train_data
origin_data.head()

,idx,class,conversation
0,0,일반 대화,"학교 점심 뭐 나와?\n주로 한식이 나와.\n학원 다녀?\n응, 영어 학원 다녀.\..."
1,1,기타 괴롭힘 대화,어이 거기 뒤뚱거리는 놈 \n나?\n그래 너 여기 뒤뚱거리는 놈이 너밖에 더 있냐?...
2,2,협박 대화,너 그따위로 운전하면 확 갈아마셔버린다.\n 뭐라구?\n 나와 이 자식아. 미안하단...
3,3,직장 내 괴롭힘 대화,길동씨 이번에 이것좀 처리해요\n이거 제가 한게 아닌데요\n팀에서 내가 니가가 어딨...
4,4,일반 대화,"비가 많이 오네.\n우산 가져왔어?\n날씨가 추워졌어.\n맞아, 이제 겨울이야.\n..."


In [8]:
train_data.sample(10)

,idx,class,conversation
3602,3602,기타 괴롭힘 대화,어휴 야 넌 왤케 뚱뚱하냐?\n내가 뚱뚱한게 뭐 어때서\n야 보기만해도 역겨워\n내...
4173,4173,협박 대화,어머나 변태인가봐\n이익\n진짜 이상한 학생을 만났네 기분이 이상해서 버스에서 빨리...
1441,1441,협박 대화,뭘 봐 미친새끼야.\n네.?\n왜 쳐다보냐고.\n.\n아저씨 왜 이러세요.\n다들 ...
1730,1730,직장 내 괴롭힘 대화,너 남아서 태권도장 청소하고 가라.\n 저번 달에도 제가 다 청소했습니다.\n 아 ...
1781,1781,직장 내 괴롭힘 대화,영희씨 이거밖에 안돼?\n죄송합니다\n똑바로 좀 하자\n네\n옷은 그게 뭐야\n네?...
4257,4257,협박 대화,아니 끼어든 것은 아저씨잖아요.\n 참나 눈을 감고 운전하셨나?\n 눈을 감기는. ...
3729,3729,갈취 대화,야 이자식 명품입었네? 벗어봐. 나도 사려고하던건데 입어만보자\n네.? 저 이거 벗...
1387,1387,직장 내 괴롭힘 대화,야 안인턴 너가 일처리를 그 따위로 하니까 내가 부장님한테 혼나잖아\n아니 전 그냥...
4,4,일반 대화,"비가 많이 오네.\n우산 가져왔어?\n날씨가 추워졌어.\n맞아, 이제 겨울이야.\n..."
595,595,직장 내 괴롭힘 대화,김 비서 오늘 저녁에 뭐해?\n 네??\n 특별한 일 없으면 밥이나 먹을까?\n 아...


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback
from sklearn.preprocessing import LabelEncoder

In [ ]:
wandb.login(key = '746fb761ab2f1b53db2dafef7340caad69224513')

In [ ]:
sweep_config = {
    "name": "sweep_test_nlp",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "learning_rate" : {
            "min" : 0.001,
            "max" : 0.1
            },
        "epoch" : {
            "distribution" : "int_uniform",
            "min" : 5,
            "max" : 10
            }
                    
        }
    }

In [2]:
from  tensorflow import keras
def build_model():
    model=keras.models.Sequential()
    model.add(keras.layers.Embedding(config.vocab, config.embeddings))
    model.add(keras.layers.GRU(units = config.units_1, return_sequences = True))
    model.add(keras.layers.GRU(units = config.units_2))
    model.add(keras.layers.Dense(config.units_3, activation='relu'))
    model.add(keras.layers.Dense(config.class_num, activation='softmax'))

    return model
    

In [3]:
def train():
    default_config = {
        "vocab" : 1000,
        "embeddings" : 64,
        "units_1" : 256,
        "units_2" : 256,
        "units_3" : 1024,
        "class_num" : 4,
        "learning_rate" : 0.005,
        "optimizer" : "adam",
        "loss" : "sparse_categorical_crossentropy",
        "metrics" : ["accuracy"],
        "epoch" : 5,
        "batch_size" : 32
    }

    wandb.init(config = default_config)
    config = wandb.config

    # Model
    model = build_model()
    
    # 머신 러닝 학습때 여러가지 optimzier를 사용할 경우나 learning rate를 조절할 경우에는 아래와 같은 형태의 코드를 응용합니다.

    if config.optimizer == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate = config.learning_rate)
    
    model.compile(optimizer = optimizer,
                  loss = config.loss,
                  metrics = config.metrics)

    # WandbCallback 함수는 후술합니다.
    
    model.fit(X_train, y_train,
              epochs = config.epoch,
              batch_size = config.batch_size,
              validation_data = (X_val, y_val),
              callbacks = [WandbCallback()])
    
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
    
    # wandb.log 함수 안에 기록하고 싶은 정보를 담습니다.
    
    wandb.log({"Test Accuracy Rate: " : round(test_accuracy * 100, 2),
               "Test Error Rate: " : round((1 - test_accuracy) * 100, 2)})